In [1]:
from ebmdatalab import bq
from ebmdatalab import maps
import pandas as pd


In [3]:
sql = """
WITH
  practice_count AS (
  SELECT
    ccg_id,
    COUNT(*) AS total_practices
  FROM
    hscic.practices
  WHERE
    setting = 4
    AND status_code = 'A'
  GROUP BY
    ccg_id)
SELECT
  TRIM(Principal_Supplier) AS supplier,
  practices.ccg_id AS pct,
  COUNT(*) AS count,
  total_practices
FROM
  hscic.vendors software
JOIN
  hscic.practices
ON
  practices.code = software.ODS
JOIN
  practice_count
ON
  practice_count.ccg_id = practices.ccg_id
WHERE
  practices.setting = 4
  AND practices.status_code = 'A'
  AND Date = '2018-12-01' ###latest data is December
GROUP BY
  supplier,
  practices.ccg_id, total_practices
  
  """

ehr_by_ccg = bq.cached_read(sql, csv_path="ehr_by_ccg.csv", use_cache=False)

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas\io\gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)


In [6]:

ehr_by_ccg['percent'] = (ehr_by_ccg['count'] / ehr_by_ccg['total_practices'] * 100).round()

In [9]:
import importlib
importlib.reload(maps)
for supplier in ehr_by_ccg.supplier.unique():
    if not supplier:
        continue
    plt = maps.ccg_map(
        ehr_by_ccg[ehr_by_ccg['supplier'] == supplier], 
        title=supplier, column='percent', 
        separate_london=False)
    plt.show()

BaseException: Data contains CCG codes we don't know about: ['112']